# Análisis Ajustado de Imágenes con DeepForest
Este notebook aplica correcciones de color (BGR→RGB), configura parámetros de sliding window (patch_size y patch_overlap) basados en el diámetro de las copas (30–50 px) y prueba tres umbrales de confianza. Además, calcula métricas de evaluación (precisión, recall, F1) y guarda resultados y las imágenes anotadas.

## 1. Clonar repositorio y configuración inicial

In [ ]:
import os
import git

# Parámetros de repositorio y carpetas
REPO_URL = 'https://github.com/maraosoc/citrus3-detector.git'
REPO_DIR = 'citrus3-detector'
DATA_DIR = os.path.join(REPO_DIR, 'data', 'samples')
RESULT_DIR = 'resultados_citrus_ajustado'

# Clonar si no existe
if not os.path.exists(REPO_DIR):
    print('Clonando repositorio...')
    git.Repo.clone_from(REPO_URL, REPO_DIR)
else:
    print('El repositorio ya existe localmente.')


## 2. Configuración de modelo, sliding window y umbrales

In [ ]:
from deepforest import main

# Cargar modelo pre-entrenado y parámetros de sliding window
modelo = main.deepforest()
modelo.use_release()

# Basado en diámetro de copa (30–50 px), definimos patch_size y patch_overlap
modelo.config['patch_size'] = 512      # tamaño de parche en px
modelo.config['patch_overlap'] = 0.5   # solape del 50%

# Lista de umbrales de confianza a probar (score_thresh)
UMBRAL_ES = [0.05, 0.1, 0.3]


## 3. Funciones auxiliares: IoU, emparejamiento y métricas

In [ ]:
import numpy as np

def compute_iou(box1, box2):
    """Calcula el IoU entre dos cajas [xmin, ymin, xmax, ymax]."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0

def match_boxes(predictions, ground_truth, iou_threshold=0.4):
    """Empareja predicciones y ground truth según IoU y devuelve TP, FP, FN."""
    matched_gt = set()
    tp = 0
    for _, pred in predictions.iterrows():
        pred_box = [pred.xmin, pred.ymin, pred.xmax, pred.ymax]
        for idx, true in ground_truth.iterrows():
            if idx in matched_gt:
                continue
            true_box = [true.xmin, true.ymin, true.xmax, true.ymax]
            if compute_iou(pred_box, true_box) >= iou_threshold:
                tp += 1
                matched_gt.add(idx)
                break
    fp = len(predictions) - tp
    fn = len(ground_truth) - tp
    return tp, fp, fn

def compute_metrics(tp, fp, fn):
    """Calcula precisión, recall y F1."""
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1


## 4. Procesamiento con corrección de color, resoluciones y umbrales

In [ ]:
import cv2
import pandas as pd
from deepforest.visualize import plot_predictions

# Definición de resoluciones
RESOLUCIONES = {
    'original': None,             # tamaño original
    'medio': (1033, 939),
    'pequeno': (516, 469)
}

os.makedirs(RESULT_DIR, exist_ok=True)
resumen = []

for img_file in os.listdir(DATA_DIR):
    if not img_file.lower().endswith(('.tif', '.png', '.jpg')):
        continue
    img_path = os.path.join(DATA_DIR, img_file)
    basename = os.path.splitext(img_file)[0]
    # Leer y convertir BGR → RGB
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Cargar ground truth si existe
    gt_path = os.path.join(REPO_DIR, 'data', 'annotations', f'{basename}_gt.csv')
    if os.path.exists(gt_path):
        ground_truth = pd.read_csv(gt_path)
    else:
        ground_truth = None

    for res_name, size in RESOLUCIONES.items():
        # Redimensionar si aplica
        if size:
            img_resized = cv2.resize(img_rgb, size, interpolation=cv2.INTER_AREA)
        else:
            img_resized = img_rgb.copy()

        for score_thresh in UMBRAL_ES:
            modelo.config['score_thresh'] = score_thresh
            # Predicción
            boxes = modelo.predict_image(image=img_resized, return_plot=False)
            boxes = boxes[boxes.score >= score_thresh]

            # Evaluación
            if ground_truth is not None:
                tp, fp, fn = match_boxes(boxes, ground_truth)
                precision, recall, f1 = compute_metrics(tp, fp, fn)
            else:
                tp = fp = fn = None
                precision = recall = f1 = None

            # Guardar CSV de predicciones
            boxes.to_csv(
                os.path.join(RESULT_DIR, f'cajas_{res_name}_th{score_thresh}_{basename}.csv'),
                index=False
            )
            # Guardar imagen anotada (convertir RGB→BGR para cv2)
            annotated = plot_predictions(img_resized, boxes)
            annotated_bgr = cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR)
            cv2.imwrite(
                os.path.join(RESULT_DIR, f'annotated_{res_name}_th{score_thresh}_{basename}.png'),
                annotated_bgr
            )

            # Agregar al resumen
            resumen.append({
                'Imagen': basename,
                'Resolución': res_name,
                'Score_thresh': score_thresh,
                'Detections': len(boxes),
                'TP': tp, 'FP': fp, 'FN': fn,
                'Precision': precision, 'Recall': recall, 'F1': f1
            })

# Guardar resumen general
import pandas as pd
df_resumen = pd.DataFrame(resumen)
df_resumen.to_csv(os.path.join(RESULT_DIR, 'resumen_general_ajustado.csv'), index=False)


## 5. Resultados
- Los CSV de predicciones y las imágenes anotadas están en `resultados_citrus_ajustado`.
- El resumen con métricas para cada umbral/resolución está en `resumen_general_ajustado.csv`.

Puedes descargar este notebook ajustado aquí:
[Descargar Notebook Ajustado](sandbox:/mnt/data/analisis_de_arboles_ajustado.ipynb)